In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import datasets
import matplotlib.pyplot as plt
from __future__ import division, print_function
import progressbar
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder

from typing import Union

np.random.seed(130301) # tạo seed để có thể reproduce kết quả

In [ ]:
def to_categorical(x, n_col=None):
    # Mình có thể dùng One-hot Encoder của Sklearn để làm việc tương tự
    # tuy nhiên việc dùng encoder mình cần phải fit_transform và khởi tạo encoder
    # mình thấy là làm vậy hơi loằng ngoằng nên mình viết hàm này để transform thẳng :D
    # các bạn có thể lựa chọn One-hot Encoder của sklearn hay thậm chí hàm
    # to_categorical của Tensorflow.
    if not n_col:
        n_col = np.amax(x) + 1
    one_hot = np.zeros((x.shape[0], n_col))
    one_hot[np.arange(x.shape[0]), x] = 1
    return one_hot

def standardize(X):
    # Tương tự với hàm to_categorical, có khá nhiều implementation của các thư viện
    # tuy vậy mình không thấy nó tiện lợi lắm nên mình tạo thành một func riêng :D. 
    # Các bạn có thể sử dụng các implementation khác!
    X_std = X
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    for col in range(np.shape(X)[1]):
        if std[col]:
            X_std[:, col] = (X_std[:, col] - mean[col]) / std[col]
    # X_std = (X - X.mean(axis=0)) / X.std(axis=0)
    return X_std

In [ ]:
class Loss:
    def __init__(self, regression: bool=True):
        self.reg = regression

    def loss(self, y_true: np.array, y_pred: np.array):
        return self._regression_loss(y_true, y_pred) if self.reg else self._classification_loss(y_true, y_pred)

    def gradient(self, y_true: np.array, y_pred: np.array):
        return self._regression_gradient(y_true, y_pred) if self.reg else self._classification_gradient(y_true, y_pred)

    def acc(self, y_true: np.array, y_pred: np.array):
        return 0 if self.reg else accuracy_score(np.argmax(y, axis=1), np.argmax(p, axis=1))

    def _regression_loss(self, y_true: np.array, y_pred: np.array):
        """ Squared Loss"""
        return 0.5 * np.power((y_true - y_pred), 2)

    def _regression_gradient(self, y_true: np.array, y_pred: np.array):
        return -(y_true - y_pred)

    def _classification_loss(self, y_true: np.array, y_pred: np.array):
        """ Categorical Cross-Entropy Loss"""
        # limit range of p to avoid division by zero
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        return - y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred)

    def _classification_gradient(self, y_true: np.array, y_pred: np.array):
        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15) 
        return - (y_true / y_pred) + (1 - y_true) / (1 - y_pred)


class GradientBoosting:
    def __init__(self, 
                n_estimators: int = 100, 
                learning_rate: float = 0.1,
                min_samples_split: Union[int, float] = 2,
                min_impurity_decrease: float = 0, 
                max_depth: int = 8, 
                regression = True):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.max_depth = max_depth
        self.regression = regression
        
        self.loss = Loss(self.regression)

        tree = DecisionTreeRegressor(
                min_samples_split=self.min_samples_split,
                min_impurity_decrease=min_impurity_decrease,
                max_depth=self.max_depth)
        
        self.trees = [tree for _ in range(n_estimators)] 


    def fit(self, X, y):
        y_pred = np.full(np.shape(y), np.mean(y, axis=0))
        for i in range(self.n_estimators):
            gradient = self.loss.gradient(y, y_pred)
            # self.loss_history.append(self.loss.loss(y, y_pred))
            self.trees[i].fit(X, gradient)
            update = self.trees[i].predict(X)
            # Update y 
            y_pred -= np.multiply(self.learning_rate, update)

    def predict(self, X):
        y_pred = np.array([])
        # Make predictions
        for tree in self.trees:
            update = tree.predict(X)
            update = np.multiply(self.learning_rate, update)
            y_pred = -update if not y_pred.any() else y_pred - update

        if not self.regression:
            y_pred = np.exp(y_pred) / np.expand_dims(np.sum(np.exp(y_pred), axis=1), axis=1)
            y_pred = np.argmax(y_pred, axis=1)
        return y_pred


class GBDT(GradientBoosting):
    def __init__(self, n_estimators=2000, learning_rate=0.05, min_samples_split=2,
                 min_info_gain=1e-7, max_depth=8, debug=False):
        super(GBDT, self).__init__(n_estimators=n_estimators, 
            learning_rate=learning_rate, 
            min_samples_split=min_samples_split, 
            min_impurity_decrease=min_info_gain,
            max_depth=max_depth,
            regression=False)

    def fit(self, X, y):
        y = to_categorical(y)
        super(GBDT, self).fit(X, y)


In [ ]:
df = pd.read_csv("/content/WA_Fn-UseC_-Telco-Customer-Churn.csv", engine='c')
customerID = df.customerID
df.drop(["customerID"], axis=1, inplace=True)

In [ ]:
def object_to_int(dataframe_series):
    if dataframe_series.dtype=='object':
        dataframe_series = LabelEncoder().fit_transform(dataframe_series)
    return dataframe_series

df = df.apply(lambda x: object_to_int(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   int64  
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   int64  
 3   Dependents        7043 non-null   int64  
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   int64  
 6   MultipleLines     7043 non-null   int64  
 7   InternetService   7043 non-null   int64  
 8   OnlineSecurity    7043 non-null   int64  
 9   OnlineBackup      7043 non-null   int64  
 10  DeviceProtection  7043 non-null   int64  
 11  TechSupport       7043 non-null   int64  
 12  StreamingTV       7043 non-null   int64  
 13  StreamingMovies   7043 non-null   int64  
 14  Contract          7043 non-null   int64  
 15  PaperlessBilling  7043 non-null   int64  
 16  PaymentMethod     7043 non-null   int64  


In [ ]:
X = df.drop(columns = ['Churn'])
y = df['Churn'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.30, random_state = 40, stratify=y)

In [ ]:
cls = GBDT()
cls.fit(X_train.values, y_train)
test_pred = cls.predict(X_test.values)
print(classification_report(test_pred, y_test))

              precision    recall  f1-score   support

           0       0.80      0.85      0.82      1462
           1       0.61      0.52      0.56       651

    accuracy                           0.75      2113
   macro avg       0.70      0.69      0.69      2113
weighted avg       0.74      0.75      0.74      2113



In [ ]:
#@title # Test với các learning rate khác nhau {"run": "auto", "display-mode": "form"}

lr = 0.034 #@param {type:"slider", min:0, max:1, step:0.0005}

cls = GBDT(learning_rate=lr)
cls.fit(X_train.values, y_train)
test_pred = cls.predict(X_test.values)
print(classification_report(test_pred, y_test))

              precision    recall  f1-score   support

           0       0.82      0.79      0.80      1619
           1       0.39      0.44      0.41       494

    accuracy                           0.71      2113
   macro avg       0.60      0.61      0.61      2113
weighted avg       0.72      0.71      0.71      2113

